In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/BachelorThesis/CookBERT/finetuning_for_downstream_tasks/question_answering/

Mounted at /content/drive
/content/drive/MyDrive/BachelorThesis/CookBERT/finetuning_for_downstream_tasks/question_answering


In [2]:
!pip install scikit-posthocs researchpy

In [3]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from scipy.stats import f_oneway # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html
import scikit_posthocs as sp # https://scikit-posthocs.readthedocs.io/en/latest/intro/
import researchpy as rp
import json

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
models = ["CookBERT", "FoodBERT", "bert-base-uncased"]
folds = 10 # number of folds for cross validation

result_df = pd.DataFrame()


for model in models:
  for fold in range(folds):
    try:
      with open(f'model_output/{model}/predict_fold_{fold}_results.json') as f:
        fold_result = json.load(f)
        result = {
            'model': model,
            'fold': fold,
            'f1': fold_result['test_f1'],
            'exact_match': fold_result['test_exact_match'],
            'samples': fold_result['predict_samples']
        }
        result_df = result_df.append(result, ignore_index=True)
    except: # if file not found just continue with the next one
      continue

In [ ]:
result_df

In [7]:
anova = f_oneway(result_df['f1'][result_df['model'] == 'FoodBERT'],
                  result_df['f1'][result_df['model'] == 'bert-base-uncased'],
                  result_df['f1'][result_df['model'] == 'CookBERT']
                 )

print(anova)
no_context_model_posthoc = sp.posthoc_ttest(result_df, val_col='f1', group_col='model',p_adjust='bonferroni')
display(no_context_model_posthoc)
display(rp.summary_cont(result_df['f1'].groupby(result_df['model'])))

F_onewayResult(statistic=26.18273367584176, pvalue=4.768529935718259e-07)


,CookBERT,FoodBERT,bert-base-uncased
CookBERT,1.000000,0.000558,0.072057
FoodBERT,0.000558,1.000000,0.000003
bert-base-uncased,0.072057,0.000003,1.000000


,N,Mean,SD,SE,95% Conf.,Interval
model,,,,,,
CookBERT,10,30.6353,1.5938,0.5040,29.4952,31.7754
FoodBERT,10,27.5065,1.3882,0.4390,26.5135,28.4996
bert-base-uncased,10,32.3945,1.5987,0.5056,31.2508,33.5382


In [ ]:
result_df.groupby(['model']).agg({'exact_match':['mean'],
                                    'f1':['mean']})

,exact_match,f1
,mean,mean
model,,
CookBERT,12.509578,30.635280
FoodBERT,10.806192,27.506528
bert-base-uncased,14.059678,32.394497


In [ ]:
# confidence intervals for exact match score
display(rp.summary_cont(result_df['exact_match'].groupby(result_df['model'])))

,N,Mean,SD,SE,95% Conf.,Interval
model,,,,,,
CookBERT,10,12.5096,1.2670,0.4007,11.6032,13.4160
FoodBERT,10,10.8062,1.0318,0.3263,10.0681,11.5443
bert-base-uncased,10,14.0597,1.4901,0.4712,12.9938,15.1256
